# 使用 PINN 进行复合材料棒中的稳态一维扩散

在本笔记中，我们将使用 PINN 求解扩散偏微分方程，然后将其扩展为求解参数化偏微分方程。我们将看到使用 Modulus API 将单个正向模拟转换为参数化模拟是多么容易（参数化可以针对边界条件、偏微分方程系数、几何形状等）。

在继续之前，让我们快速回顾一下 PINN 的理论。我们将把问题的物理原理嵌入到神经网络中，在这种情况下，我们可以利用它们来近似求解给定微分方程和边界条件的解，而无需任何来自其他求解器的训练数据。更具体地说，神经网络将被训练以最小化由微分方程和边界条件形成的损失函数。如果网络能够最小化这个损失，那么它实际上就解决了给定的微分方程。更多关于物理信息神经网络 (PINN) 的信息，请参阅 Raissi 等人发表的[这篇论文](https://www.sciencedirect.com/science/article/pii/S0021999118307125?casa_token=1CnSbVeDwJ0AAAAA:-8a6ZMjO7RgYjFBAxIoVU2sWAdsqFzLRTNHA1BkRryNPXx5Vjc8hCDCkS99gcZR0B1rpa33a30yG)。

在本笔记中，我们将求解复合材料棒中的稳态一维传热问题。您可以参考 [Modulus 用户文档](https://docs.nvidia.com/deeplearning/modulus/modulus-v2209/index.html)，了解更多使用 Modulus 求解不同类型偏微分方程的示例。

## 学习成果
- 如何使用 Modulus 模拟基于 PINN 的物理问题
- 如何编写自己的偏微分方程并计算不同的损失函数
- 如何使用构造性立体几何 (CSG) 模块
- 如何使用 Modulus 求解参数化偏微分方程

## 问题描述

我们的目标是获取由两种导热系数不同的材料制成的棒状物体内部的温度分布。该问题的几何形状和具体问题描述如下：

<img src="diffusion_bar_geometry.png" alt="Drawing" style="width: 600px;"/>

复合材料棒从 $x=0$ 延伸至 $x=2$。复合材料棒在 $x=0$ 到 $x=1$ 处的导热系数为 $D_1=10$，在 $x=1$ 到 $x=2$ 处的导热系数为 $D_2=0.1$。复合材料棒的两端，$x=0$ 和 $x=2$ 分别保持在恒定的 $0$ 和 $100$ 温度。为了简化建模，我们将复合材料棒视为两个独立的棒状物体，棒1 和 棒2，它们的两端连接在一起。我们将 棒1 中的温度视为 $U_1$，棒2 中的温度视为 $U_2$。

控制该问题的方程和边界条件可以用数学形式表示为...

棒1 和 棒2 中的一维温度扩散：

$$
\begin{align}
\frac{d}{dx}\left( D_1\frac{dU_1}{dx} \right) = 0, && \text{when } 0<x<1 \\
\frac{d}{dx}\left( D_2\frac{dU_2}{dx} \right) = 0, && \text{when } 1<x<2 \\
\end{align}
$$

界面 $(x=1)$ 处的通量和温度连续性
$$
\begin{align}
D_1\frac{dU_1}{dx} = D_2\frac{dU_2}{dx}, && \text{when } x=1 \\
U_1 = U_2, && \text{when } x=1 \\
\end{align}
$$

## 案例设置

现在我们已经定义了问题，让我们来看看使用 Modulus 求解该问题所需的代码。Modulus 提供了各种辅助函数/API，可以帮助我们轻松地设置问题。Modulus 提供了一些 API，可以使用构造实体几何 (CSG) 模块以参数化方式对几何体进行建模，以用户友好的符号格式编写所需的方程，并且还提供了几种高级神经网络架构供选择，以解决更复杂的问题。

现在，让我们通过导入所需的库和包来开始求解该问题。

## <span style='color:Red'> 注意！</span>

本笔记本将介绍 [`diffusion_bar.py`](../../source_code/diffusion_1d/diffusion_bar.py) 脚本的内容。因此，本笔记本中的大部分代码块仅供参考。您需要更新主 [`diffusion_bar.py`](../../source_code/diffusion_1d/diffusion_bar.py) 脚本才能进行任何更改或进行超参数调整。如果您更喜欢更具交互性的 Jupyter 设置方式，请参阅 [Modulus 项目剖析](../introduction/Modulus_Anatomy.ipynb) 了解更多信息。

```python
import torch
import numpy as np
from sympy import Symbol, Eq, Function, Number

import modulus
from modulus.hydra import to_absolute_path, ModulusConfig, instantiate_arch
from modulus.solver import Solver
from modulus.domain import Domain
from modulus.geometry.primitives_1d import Line1D
from modulus.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
)
from modulus.domain.validator import PointwiseValidator
from modulus.domain.monitor import PointwiseMonitor
from modulus.key import Key
from modulus.node import Node
from modulus.eq.pde import PDE
```

从 v22.03 开始，Modulus 支持用于解决数据驱动问题（数据以网格/网格形式提供）以及使用点云进行问题建模的 PINN 问题（PINN 问题）的 API。在本节中，由于我们将使用 PINN 方法求解一维扩散问题，因此我们将使用 CSG 模块设置几何体和点云。`Solver` 类用于训练和评估 Modulus 的神经网络求解器。`Domain` 类用于为问题添加约束、监视器、验证器等。

`geometry` 模块包含可用于定义问题的预定义几何体。我们将在代码的后续部分详细介绍每个几何体。有关所有不同模块的更多详细信息，建议您参考 [Modulus API 文档](https://docs.nvidia.com/deeplearning/modulus/modulus-v2209/api/api_index.html)。

## 创建几何体

在本题中，我们将使用 `geometry` 模块中的 [`Line1D`](https://docs.nvidia.com/deeplearning/modulus/modulus-v2209/api/modulus.geometry.html#modulus.geometry.primitives_1d.Line1D) 类创建一个一维几何体。该模块还包含一些二维和三维形状，例如矩形、圆形、三角形、长方体、球体、圆环、圆锥体、四面体等。我们将使用两个端点来定义一维线对象。对于复合条，我们将按照问题描述中的定义创建两个独立的条。

```python
# params for domain
L1 = Line1D(0,1)
L2 = Line1D(1,2)
```

接下来，我们将定义问题的属性，这些属性稍后将在建立方程、边界条件等时使用。对于这个特定的问题，我们可以分析找到界面处的温度，并将使用它来形成验证数据，以便与我们的神经网络结果进行比较。

```python
D1 = 1e1
D2 = 1e-1

Tc = 100
Ta = 0
Tb = (Tc + (D1 / D2) * Ta) / (1 + (D1 / D2))

print(Ta)
print(Tb)
print(Tc)
```

## 定义问题的微分方程

`PDE` 类允许我们在 SymPy 中以符号形式编写方程。这使我们能够以最自然的方式快速编写方程。SymPy 方程在后端转换为 PyTorch 表达式，并且可以打印以确保正确实现。

Modulus 还预定义了几个常见的 PDE 供我们选择。PDEs 模块中已经包含的一些 PDE 包括：纳维-斯托克斯方程、线性弹性方程、平流扩散方程、波动方程等。

让我们创建 PDE 来定义扩散方程。我们将以最通用的瞬态三维形式定义该方程，然后使用参数“dim”将其降维为低维形式。
$$\frac{\partial T}{\partial t}= \nabla\cdot \left( D \nabla T \right) + Q$$

让我们从继承 `PDE` 类开始定义方程。我们将为该类创建初始化方法，用于定义我们感兴趣的方程。我们将使用源（`Q`）、扩散率（`D`）和扩散符号（`T`）来定义扩散方程。如果 `D` 或 `Q` 以字符串形式给出，我们将将其转换为函数形式。这将使我们能够求解具有空间/时间变化特性的问题。

```python
class Diffusion(PDE):
    name = "Diffusion"

    def __init__(self, T="T", D="D", Q=0, dim=3, time=True):
        # set params
        self.T = T
        self.dim = dim
        self.time = time

        # coordinates
        x, y, z = Symbol("x"), Symbol("y"), Symbol("z")

        # time
        t = Symbol("t")

        # make input variables
        input_variables = {"x": x, "y": y, "z": z, "t": t}
        if self.dim == 1:
            input_variables.pop("y")
            input_variables.pop("z")
        elif self.dim == 2:
            input_variables.pop("z")
        if not self.time:
            input_variables.pop("t")

        # Temperature
        assert type(T) == str, "T needs to be string"
        T = Function(T)(*input_variables)

        # Diffusivity
        if type(D) is str:
            D = Function(D)(*input_variables)
        elif type(D) in [float, int]:
            D = Number(D)

        # Source
        if type(Q) is str:
            Q = Function(Q)(*input_variables)
        elif type(Q) in [float, int]:
            Q = Number(Q)

        # set equations
        self.equations = {}
        self.equations["diffusion_" + self.T] = (
            T.diff(t)
            - (D * T.diff(x)).diff(x)
            - (D * T.diff(y)).diff(y)
            - (D * T.diff(z)).diff(z)
            - Q
        )
```

让我们回顾一下这个类。首先，我们用 SymPy 符号定义了输入变量 $x, y, z$ 和 $t$。然后，我们定义了依赖于输入变量 $(x, y, z, t)$ 的 $T$、$D$ 和 $Q$ 函数。利用这些函数，我们可以写出简单的方程 $T_t = \nabla \cdot (D \nabla T) + Q$。我们将这个方程添加到 `equations` 字典中，从而将其存储在类中。

请注意，我们将偏微分方程的所有项移动到了左侧或右侧。这样，在使用约束中的方程时，我们可以将自定义源函数赋给 `'diffusion_T'` 键而不是 0，从而为我们的偏微分方程添加更多源项。

太棒了！我们刚刚用 Modulus 编写了自己的偏微分方程。一旦您理解了编写简单偏微分方程的流程，就可以轻松地将其扩展到不同的偏微分方程中。您还可以通过向方程字典添加新键，将多个偏微分方程 (PDE) 捆绑到同一个文件中。下方展示了界面边界条件的代码，其中需要保持场（狄利克雷）和通量（诺伊曼）连续性。更多编写您自己的偏微分方程的示例，请参阅 [Modulus 用户文档](https://docs.nvidia.com/deeplearning/modulus/modulus-v2209/user_guide/foundational/1d_wave_equation.html)。

**注意**：由于我们要求解两个杆中的两种不同温度，因此需要场连续性条件。

```python
class DiffusionInterface(PDE):
    name = "DiffusionInterface"

    def __init__(self, T_1, T_2, D_1, D_2, dim=3, time=True):
        # set params
        self.T_1 = T_1
        self.T_2 = T_2
        self.dim = dim
        self.time = time

        # coordinates
        x, y, z = Symbol("x"), Symbol("y"), Symbol("z")
        normal_x, normal_y, normal_z = (
            Symbol("normal_x"),
            Symbol("normal_y"),
            Symbol("normal_z"),
        )

        # time
        t = Symbol("t")

        # make input variables
        input_variables = {"x": x, "y": y, "z": z, "t": t}
        if self.dim == 1:
            input_variables.pop("y")
            input_variables.pop("z")
        elif self.dim == 2:
            input_variables.pop("z")
        if not self.time:
            input_variables.pop("t")

        # Diffusivity
        if type(D_1) is str:
            D_1 = Function(D_1)(*input_variables)
        elif type(D_1) in [float, int]:
            D_1 = Number(D_1)
        if type(D_2) is str:
            D_2 = Function(D_2)(*input_variables)
        elif type(D_2) in [float, int]:
            D_2 = Number(D_2)

        # variables to match the boundary conditions (example Temperature)
        T_1 = Function(T_1)(*input_variables)
        T_2 = Function(T_2)(*input_variables)

        # set equations
        self.equations = {}
        self.equations["diffusion_interface_dirichlet_" + self.T_1 + "_" + self.T_2] = (
            T_1 - T_2
        )
        flux_1 = D_1 * (
            normal_x * T_1.diff(x) + normal_y * T_1.diff(y) + normal_z * T_1.diff(z)
        )
        flux_2 = D_2 * (
            normal_x * T_2.diff(x) + normal_y * T_2.diff(y) + normal_z * T_2.diff(z)
        )
        self.equations["diffusion_interface_neumann_" + self.T_1 + "_" + self.T_2] = (
            flux_1 - flux_2
        )
```

## 创建 PDE 和神经网络节点

默认的 [`FullyConnectedArch`](https://docs.nvidia.com/deeplearning/modulus/modulus-v2209/api/modulus.models.html#module-modulus.models.fully_connected)（通过在 `instantiate_arch()` 函数的 `cfg` 参数中设置 `cfg.arch.fully_connected` 来选择）可用于创建针对此问题的神经网络。默认的 `FullyConnectedArch` 代表一个 6 层 MLP（多层感知器）架构，每层包含 512 个节点，并使用 swish (SiLU) 作为激活函数。所有这些参数均可由用户配置，并可使用 Hydra 配置文件进行配置。定义完所有 PDE 和架构后，我们可以创建一个节点列表，用于传递给我们想要满足此问题的不同约束（例如方程残差、边界条件等）。

这里我们创建一个包含该问题所有计算节点的列表。当此列表传递给每个类的 `nodes` 参数时，Modulus 将根据 `outvar` 目录中指定的变量自动展开/评估相应的节点。

我们将在 `run` 函数中定义该问题的所有代码，如下所示：

```python
@modulus.main(config_path="conf", config_name="config")
def run(cfg: ModulusConfig) -> None:
    # make list of nodes to unroll graph on
    diff_u1 = Diffusion(T="u_1", D=D1, dim=1, time=False)
    diff_u2 = Diffusion(T="u_2", D=D2, dim=1, time=False)
    diff_in = DiffusionInterface("u_1", "u_2", D1, D2, dim=1, time=False)

    diff_net_u_1 = instantiate_arch(
        input_keys=[Key("x")],
        output_keys=[Key("u_1")],
        cfg=cfg.arch.fully_connected,
    )
    diff_net_u_2 = instantiate_arch(
        input_keys=[Key("x")],
        output_keys=[Key("u_2")],
        cfg=cfg.arch.fully_connected,
    )

    nodes = (
        diff_u1.make_nodes()
        + diff_u2.make_nodes()
        + diff_in.make_nodes()
        + [diff_net_u_1.make_node(name="u1_network", jit=cfg.jit)]
        + [diff_net_u_2.make_node(name="u2_network", jit=cfg.jit)]
    )
```

### 关于使用 Hydra 配置 Modulus 的简要说明

使用 Modulus 的核心是 Hydra 配置文件。本示例的配置文件如下所示。使用 Hydra 可以以高度可定制且用户友好的方式配置 Modulus 的大部分功能。更多信息请参阅 [Modulus 配置](https://docs.nvidia.com/deeplearning/modulus/modulus-v2209/user_guide/features/configuration.html)。

```yaml
defaults :
  - modulus_default
  - arch:
      - fully_connected
  - scheduler: tf_exponential_lr
  - optimizer: adam
  - loss: sum
  - _self_

arch:
    fully_connected:
        layer_size: 256

save_filetypes : "vtk,npz"

scheduler:
  decay_rate: 0.95
  decay_steps: 100

optimizer: 
  lr : 1e-4

training:
  rec_results_freq: 1000
  max_steps : 5000

batch_size:
  rhs: 2
  lhs: 2
  interface: 2
  interior_u1: 200
  interior_u2: 200
```

## 设置域：指定边界和偏微分方程约束

如前所述，我们需要定义一个用于训练神经网络的域。使用 `Solver` 类时，`Domain` 和配置将作为输入传递。除了约束之外，您还可以向 `Domain` 添加各种其他实用程序，例如监视器、验证数据、用于推理的点等等。

```python
    # make domain add constraints to the solver
    domain = Domain()
```

现在，让我们研究如何向该域添加约束，以驱动神经网络优化。重申一下，对于这个物理驱动的问题，这些约束是边界条件和方程残差。目标是精确满足边界条件，理想情况下使偏微分方程残差趋于 0。这些约束可以在 Modulus 中使用 `PointwiseBoundaryConstraint` 和 `PointwiseInteriorConstraint` 等类来指定。更多信息，请参阅 [入门笔记本](../introduction/Introductory_Notebook.ipynb)。

**边界约束**：可以使用 `PointwiseBoundaryConstraint` 类对边界进行采样。这将对我们在 `geometry` 参数中指定的几何体的整个边界进行采样，在本例中，是一维线的两个端点。可以使用 `criteria` 参数，根据特定标准对几何体的特定边界进行子采样。例如，为了对 `L1` 的左端进行采样，`criteria` 设置为 `Eq(x, 0)`。

边界条件的期望值在 `outvar` 参数中以字典形式列出。对于这个问题，在 $x=0$ 处，我们有 `'u_1':0`，在 $x=2$ 处，我们有 `'u_2':100`。在 $x=1$ 处，我们有之前定义的接口条件 `'diffusion_interface_dirichlet_u_1_u_2':0` 和 `'diffusion_interface_neumann_u_1_u_2':0`（即 $U_1=U_2$ 和 $D_1\frac{dU_1}{dx}=D_2\frac{dU_2}{dx}$）。这些字典随后会在展开计算图（使用 `nodes` 参数指定）进行训练时使用。

使用 `batch_size` 参数指定每个边界上要采样的点数。

**待求解方程**：我们定义的扩散偏微分方程 (PDE) 会作用于 `L1` 和 `L2` 两个条形内部的所有点。我们将使用 `PointwiseInteriorConstraint` 类对几何结构内部的点进行采样。同样，相应的几何结构在 `geometry` 参数中指定：待求解方程以字典形式输入到 `outvar` 参数中。这些字典随后会在展开计算图（使用 `nodes` 参数指定）进行训练时使用。

对于这个问题，我们分别为 `L1` 和 `L2` 条形设置了 `'diffusion_u_1':0` 和 `'diffusion_u_2':0`。参数 `bounds` 决定了变量 $x$ 和 $y$ 的采样范围。可选的 `lambda` 参数可用于确定不同损失函数的权重。在这个问题中，我们对每个点的损失赋予相同的权重，因此它不会被使用。

**注意**：这里我们指定了同一个列表，其中包含计算图中所有可能的节点。这样做纯粹是为了简单起见，避免为每个约束创建单独的节点列表。Modulus 会根据 `outvar` 中指定的键自动选择合适的节点进行计算，因此未使用的节点不会被评估。

```python
    # sympy variables
    x = Symbol("x")

    # right hand side (x = 2) Pt c
    rhs = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=L2,
        outvar={"u_2": Tc},
        batch_size=cfg.batch_size.rhs,
        criteria=Eq(x, 2),
    )
    domain.add_constraint(rhs, "right_hand_side")

    # left hand side (x = 0) Pt a
    lhs = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=L1,
        outvar={"u_1": Ta},
        batch_size=cfg.batch_size.lhs,
        criteria=Eq(x, 0),
    )
    domain.add_constraint(lhs, "left_hand_side")

    # interface 1-2
    interface = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=L1,
        outvar={
            "diffusion_interface_dirichlet_u_1_u_2": 0,
            "diffusion_interface_neumann_u_1_u_2": 0,
        },
        batch_size=cfg.batch_size.interface,
        criteria=Eq(x, 1),
    )
    domain.add_constraint(interface, "interface")

    # interior 1
    interior_u1 = PointwiseInteriorConstraint(
        nodes=nodes,
        geometry=L1,
        outvar={"diffusion_u_1": 0},
        bounds={x: (0, 1)},
        batch_size=cfg.batch_size.interior_u1,
    )
    domain.add_constraint(interior_u1, "interior_u1")

    # interior 2
    interior_u2 = PointwiseInteriorConstraint(
        nodes=nodes,
        geometry=L2,
        outvar={"diffusion_u_2": 0},
        bounds={x: (1, 2)},
        batch_size=cfg.batch_size.interior_u2,
    )
    domain.add_constraint(interior_u2, "interior_u2")
```

## 添加验证数据

对于这个一维条形问题，其中每个条形的电导率是恒定的，温度随固体内部位置线性变化。解析解如下：

$$
\begin{align}
U_1 = xT_b + (1-x)T_a, && \text{when } 0 \leq x \leq 1 \\
U_2 = (x-1)T_c + (2-x)T_b, && \text{when } 1 \leq x \leq 2 \\
\end{align}
$$

其中：
$$
\begin{align}
T_a = U_1|_{x=0}, && T_c = U_2|_{x=2}, && \frac{\left(T_c + \left( D_1/D_2 \right)T_a \right)}{1+ \left( D_1/D_2 \right)}\\
\end{align}
$$

现在让我们创建验证器。使用 PointwiseValidator 类将验证数据添加到域中。我们根据上面给出的解析表达式，使用 NumPy 求解 u_1 和 u_2。在构造函数的定义中，使用了由输入和输出变量组成的 NumPy 数组字典（invar_numpy 和 outvar_numpy），以及相应的节点。

```python
    # validation data
    x = np.expand_dims(np.linspace(0, 1, 100), axis=-1)
    u_1 = x * Tb + (1 - x) * Ta
    invar_numpy = {"x": x}
    outvar_numpy = {"u_1": u_1}
    val = PointwiseValidator(nodes=nodes, invar=invar_numpy, true_outvar=outvar_numpy)
    domain.add_validator(val, name="Val1")

    # make validation data line 2
    x = np.expand_dims(np.linspace(1, 2, 100), axis=-1)
    u_2 = (x - 1) * Tc + (2 - x) * Tb
    invar_numpy = {"x": x}
    outvar_numpy = {"u_2": u_2}
    val = PointwiseValidator(nodes=nodes, invar=invar_numpy, true_outvar=outvar_numpy)
    domain.add_validator(val, name="Val2")
```

## 添加监视器

Modulus 允许您在模拟过程中在 Tensorboard 中监视所需量并评估收敛性。[`PointwiseMonitor`](https://docs.nvidia.com/deeplearning/modulus/modulus-v2209/api/modulus.domain.validator.html#modulus.domain.validator.continuous.PointwiseValidator) 可用于创建此类功能。此类量的示例可以是变量的点值、表面平均值、体积平均值或任何可以使用正在求解的变量形成的派生量。

这些变量以 PyTorch 张量的形式提供。我们可以执行 PyTorch 中提供的张量运算来计算我们选择的任何所需的派生量。

在下面的代码中，我们创建了界面通量的监视器。变量 `u_1__x` 表示 `u_1` 在 x 方向上的导数（两个下划线 (`__`) 和变量 (`x`)）。使用 Modulus 库处理其他导数时，使用相同的符号。（例如，在训练域中，$\frac{dU_1}{dx}=0$ 的诺伊曼边界条件可以指定为 `'u_1__x':0`，以解决具有绝热/固定通量边界条件的相同问题。）

采样点的选择方式与我们指定一些内部约束的方式类似。

```python
    # make monitors
    invar_numpy = {"x": [[1.0]]}
    monitor = PointwiseMonitor(
        invar_numpy,
        output_names=["u_1__x"],
        metrics={"flux_u1": lambda var: torch.mean(var["u_1__x"])},
        nodes=nodes,
        requires_grad=True,
    )
    domain.add_monitor(monitor)

    monitor = PointwiseMonitor(
        invar_numpy,
        output_names=["u_2__x"],
        metrics={"flux_u2": lambda var: torch.mean(var["u_2__x"])},
        nodes=nodes,
        requires_grad=True,
    )
    domain.add_monitor(monitor)
```

## 整合所有内容：求解器和训练

我们可以使用刚刚创建的域以及定义优化器选项和设置（即 conf）的其他配置，使用 Modulus 的 `Solver` 类来创建一个求解器。然后可以使用 `solve` 方法执行该求解器。

```python
    # make solver
    slv = Solver(cfg, domain)

    # start solver
    slv.solve()
    
if __name__ == "__main__":
    run()
```

太棒了！我们刚刚用 Modulus 完成了问题的文件设置。现在我们可以使用神经网络来求解偏微分方程了。

## Tensorboard

在开始训练之前，我们可以利用 Tensorboard 来可视化我们刚刚创建的几个监视器的损失值和收敛情况。执行以下命令启动 Tensorboard。训练开始后，您可以使用 Tensorboard 中的刷新图标查看可视化结果。

In [ ]:
%load_ext tensorboard
%tensorboard --host 0.0.0.0 --logdir outputs/diffusion_bar

启动并运行 Tensorboard 进行监控后，请在 Jupyter 单元中执行 `diffusion_bar.py` 脚本。此示例已保存，下方代码块将执行该脚本 [`diffusion_bar.py`](../../source_code/diffusion_1d/diffusion_bar.py)。建议您在执行前打开脚本并仔细阅读代码。您还可以编辑模型 [`config.yaml`](../../source_code/diffusion_1d/conf/) 文件中的参数，并查看其对结果的影响。

In [ ]:
!python3 ../../source_code/diffusion_1d/diffusion_bar.py

## 可视化解决方案

Modulus 以多种格式保存数据（包括 .vtp、.vtk 和 .npz）。您可以绘制 .npz 数组来可视化模拟输出。生成的 .npz 文件位于 `outputs/` 目录中。

现在，让我们绘制解析解和神经网络解沿条形的温度曲线。绘制结果的示例脚本如下所示。训练完成后，您应该会得到如下所示的结果。正如我们所见，对于这个扩散问题，我们的神经网络解和解析解几乎完全匹配。

<img src="image_diffusion_problem_bootcamp.png" alt="Drawing" style="width: 500px;"/>

In [ ]:
%%capture
import sys
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

plt.figure()

network_dir = "./outputs/diffusion_bar/validators/"
data_1 = np.load(network_dir + "Val1.npz", allow_pickle=True)
data_2 = np.load(network_dir + "Val2.npz", allow_pickle=True)
data_1 = np.atleast_1d(data_1.f.arr_0)[0]
data_2 = np.atleast_1d(data_2.f.arr_0)[0]

plt.plot(data_1["x"][:, 0], data_1["pred_u_1"][:, 0], "--", label="u_1_pred")
plt.plot(data_2["x"][:, 0], data_2["pred_u_2"][:, 0], "--", label="u_2_pred")
plt.plot(data_1["x"][:, 0], data_1["true_u_1"][:, 0], label="u_1_true")
plt.plot(data_2["x"][:, 0], data_2["true_u_2"][:, 0], label="u_2_true")

plt.legend()
plt.savefig("image_diffusion_problem_bootcamp")

In [ ]:
from IPython.display import Image
display(Image(filename='./image_diffusion_problem_bootcamp.png'))

## 下一步

在下一个笔记本中，我们将训练相同的一维扩散问题，但这次通过参数化第一个条的电导率来实现。

请继续阅读[下一个笔记本](Diffusion_Problem_Parameterized.ipynb)。